In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np 
from sklearn.metrics import r2_score, mean_squared_error
import lightgbm as lgb
import statsmodels.api as sm

from snowflake.snowpark.context import get_active_session
from snowflake.ml.registry import Registry
session = get_active_session()
registry = Registry(session)

# 📝 Analyst Sentiment vs Market Reaction

## Sentiment Extraction  
We built an **AI-based sentiment dataset** for the **Dow Jones 30 earnings calls** (2024+).  
- **Source**: annotated Q&A transcripts from `COMPANY_EVENT_TRANSCRIPT_ATTRIBUTES`.  
- **Pre-processing**: deduplicated transcripts by ticker/timestamp; kept earliest point-in-time version.  
- **AI Scoring**: Claude (`claude-4-sonnet`) analyzed only **analyst questions/tone**, ignoring management’s scripted remarks.  
- **Output**: JSON with  
  - `sentiment_score` (1–10 scale),  
  - `reason` (brief justification),  
  - `analyst_count` (unique analysts).  
- Dropped events with ≤1 analyst to avoid company-only remarks.

## Linking Sentiment to Returns  
We tested whether analyst sentiment predicts short-term stock moves.  
- **Signal**:  
  - Level and Change (compare to prev call) of the Analyst Sentiment

- **Returns**:  
  - `1D return` = reaction during the call,  
  - `3D return` = drift after the call (`lead1+lead2+lead3`).  
- **Winsorization**: clipped returns at 1st–99th percentiles to reduce outlier influence.  
- **Regression**: ran OLS  
  \[
    \text{return} = a + b \cdot \text{sentiment}
  \]  
  and reported slope (β), t-stat, and sample size.  
- **Plots**: two-panel figure showing sentiment vs 1D and 3D returns, each with scatter, OLS fit, and β/t-stat annotation.

## Takeaway  
This workflow connects **qualitative analyst sentiment** to **quantitative market reactions**, helping evaluate whether bullish/bearish analyst tone during calls aligns with **immediate reactions** or **post-call drift** in stock prices.





In [ ]:
-- create analyst sentiment for Snowflake
-- Note timestamp is in UTC
WITH filtered_transcripts AS (
    SELECT *
    FROM SNOWFLAKE_PUBLIC_DATA_PAID.PUBLIC_DATA.COMPANY_EVENT_TRANSCRIPT_ATTRIBUTES
    WHERE primary_ticker IN ('SNOW')
-- start with Dow Jones 30 stocks 
      AND event_type = 'Earnings Call'
      AND transcript_type = 'SPEAKERS_ANNOTATED'
      AND transcript IS NOT NULL
      AND event_timestamp >= '2024-01-01'
    ORDER BY event_timestamp DESC
),

deduplicated_transcripts AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY primary_ticker, event_timestamp
            ORDER BY created_at ASC   -- keep the earliest version - point-in-time 
        ) AS rn
    FROM filtered_transcripts
),

unique_transcripts AS (
    SELECT
        primary_ticker,
        event_timestamp,
        event_type,
        created_at,
        transcript
    FROM deduplicated_transcripts
    WHERE rn = 1
),

ai_analysis AS (
    SELECT
        primary_ticker,
        event_timestamp,
        event_type,
        created_at,
        ai_complete(
            'claude-4-sonnet', -- use claude for the moment 
            CONCAT_WS('\n',
            
'You are analyzing sell side and buy side analysts sentiment in a public company''s earnings call transcript.',
                
'Focus ONLY on analyst questions, tone, and reactions to management responses.- Ignore management’s prepared remarks unless directly referenced by analysts. - Pay attention to how analysts compare results to prior earnings calls and to market expectations.',
                
'Use a CONSISTENT STANDARD across all earnings calls, regardless of sector, size, or company specifics.',
                
'Evaluate (internally, equal weight) the following aspects:',
                
'a) Guidance/Financial Outlook (revenue growth, earnings, margins, bookings, cash flow)',
                
'b) Strategy/Product/Innovation (new initiatives, products, technologies, or business models)',

'c) Competitive/Market Positioning (market share, competition, regulation, macro/sector trends)',

'd) Management Execution & Credibility (track record, transparency, quality of responses, consistency with prior guidance)',
                
'Compute ONE overall analyst sentiment score on a 1-10 scale, where:',
                
'1 = Extremely pessimistic/concerned, 5 = Neutral/mixed, 10 = Extremely optimistic/bullish.',
                
'Return your response as valid JSON with this exact format:',
                
'{"score": <integer 1-10>, "reason": <brief explanation covering the four aspects>, "analyst_count": <integer number of unique analysts>}',

'Transcript:',

transcript
            )
        ) AS ai_response
    FROM unique_transcripts
)

SELECT
    primary_ticker,
    event_timestamp,
    event_type,
    created_at,
    /* Safely parse JSON. If parsing fails, these will be NULL. */
    (TRY_PARSE_JSON(ai_response):score)::INT      AS sentiment_score,
    (TRY_PARSE_JSON(ai_response):analyst_count)::INT AS unique_analyst_count,
    (TRY_PARSE_JSON(ai_response):reason)::STRING  AS sentiment_reason

    -- , ai_response AS raw_ai_response  -- Uncomment for debugging
FROM ai_analysis
where unique_analyst_count > 1 -- remove data that only contains speakers from the public company
order by primary_ticker, event_timestamp desc; 


In [ ]:
-- create analyst sentiment for DOW JONES 30 company 
-- Note timestamp is in UTC
WITH filtered_transcripts AS (
    SELECT *
    FROM SNOWFLAKE_PUBLIC_DATA_PAID.PUBLIC_DATA.COMPANY_EVENT_TRANSCRIPT_ATTRIBUTES
    WHERE primary_ticker IN ('MMM', 'AXP', 'AMGN', 'AMZN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'GS', 'HD', 'HON', 'IBM', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PG', 'RTX', 'CRM', 'SHW', 'TRV', 'UNH', 'V', 'WMT', 'NVDA')
-- start with Dow Jones 30 stocks 
      AND event_type = 'Earnings Call'
      AND transcript_type = 'SPEAKERS_ANNOTATED'
      AND transcript IS NOT NULL
      AND event_timestamp >= '2024-01-01'
    ORDER BY event_timestamp DESC
),

deduplicated_transcripts AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY primary_ticker, event_timestamp
            ORDER BY created_at ASC   -- keep the earliest version - point-in-time 
        ) AS rn
    FROM filtered_transcripts
),

unique_transcripts AS (
    SELECT
        primary_ticker,
        event_timestamp,
        event_type,
        created_at,
        transcript
    FROM deduplicated_transcripts
    WHERE rn = 1
),

ai_analysis AS (
    SELECT
        primary_ticker,
        event_timestamp,
        event_type,
        created_at,
        ai_complete(
            'claude-4-sonnet', -- use claude for the moment 
            CONCAT_WS('\n',
                'You are analyzing sell side and buy side analysts sentiment in a public company''s earnings call transcript.',
                'Focus ONLY on analyst questions, tone, and reactions to management responses.- Ignore management’s prepared remarks unless directly referenced by analysts. - Pay attention to how analysts compare results to prior earnings calls and to market expectations.',
                'Use a CONSISTENT STANDARD across all earnings calls, regardless of sector, size, or company specifics.',
                'Evaluate (internally, equal weight) the following aspects:',
                'a) Guidance/Financial Outlook (revenue growth, earnings, margins, bookings, cash flow)',
                'b) Strategy/Product/Innovation (new initiatives, products, technologies, or business models)',
                'c) Competitive/Market Positioning (market share, competition, regulation, macro/sector trends)',
                'd) Management Execution & Credibility (track record, transparency, quality of responses, consistency with prior guidance)',
                'Compute ONE overall analyst sentiment score on a 1-10 scale, where:',
                '1 = Extremely pessimistic/concerned, 5 = Neutral/mixed, 10 = Extremely optimistic/bullish.',
                'Return your response as valid JSON with this exact format:',
                '{"score": <integer 1-10>, "reason": <brief explanation covering the four aspects>, "analyst_count": <integer number of unique analysts>}',
                'Transcript:',
                transcript
            )
        ) AS ai_response
    FROM unique_transcripts
)

SELECT
    primary_ticker,
    event_timestamp,
    event_type,
    created_at,
    /* Safely parse JSON. If parsing fails, these will be NULL. */
    (TRY_PARSE_JSON(ai_response):score)::INT      AS sentiment_score,
    (TRY_PARSE_JSON(ai_response):analyst_count)::INT AS unique_analyst_count,
    (TRY_PARSE_JSON(ai_response):reason)::STRING  AS sentiment_reason

    -- , ai_response AS raw_ai_response  -- Uncomment for debugging
FROM ai_analysis
where unique_analyst_count > 1 -- remove data that only contains speakers from the public company
order by primary_ticker, event_timestamp; 


In [ ]:
df_sentiment = cell4.to_pandas()
df_sentiment.columns = df_sentiment.columns.str.lower()
df_sentiment.rename(columns={'primary_ticker': 'ticker'}, inplace=True)
df_sentiment['event_timestamp_et'] = (
    pd.to_datetime(df_sentiment['event_timestamp'])
    .dt.tz_localize('UTC')
    .dt.tz_convert('US/Eastern')
)
print(df_sentiment[['ticker', 'event_timestamp', 'event_timestamp_et']].head(10))
df_sentiment = df_sentiment.sort_values(['ticker', 'event_timestamp_et'])
df_sentiment['sentiment_change'] = df_sentiment.groupby('ticker')['sentiment_score'].diff()
df_sentiment['sentiment_score'].describe()

Read the price data, note we should use adjusted price instead of raw!

In [ ]:
SELECT * 
FROM SNOWFLAKE_PUBLIC_DATA_PAID.PUBLIC_DATA.STOCK_PRICE_TIMESERIES 
WHERE ticker IN (
    'MMM', 'AXP', 'AMGN', 'AMZN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 
    'DIS', 'GS', 'HD', 'HON', 'IBM', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 
    'NKE', 'PG', 'RTX', 'CRM', 'SHW', 'TRV', 'UNH', 'V', 'WMT', 'NVDA'
)
  AND variable = 'post-market_close' -- make sure to use adjusted price later, waiting for the update from the data team
ORDER BY ticker, date DESC;

In [ ]:
df_price = cell11.to_pandas()
df_price.columns = df_price.columns.str.lower()
df_price.rename(columns={'value': 'price'}, inplace=True)
## assume market close at 4:00 pm, again we should update with the right time once the data is ready  
df_price['date'] = pd.to_datetime(df_price['date'])
df_price['date_time'] = pd.to_datetime(df_price['date'].astype(str) + ' 16:00:00').dt.tz_localize('US/Eastern')

In [ ]:
# # Set up the plotting style
# plt.style.use('default')
# sns.set_palette("husl")

# # Get unique tickers (use 'primary_ticker' if you renamed the column)
# ticker_list = df_price['ticker'].unique()  # or df_price['ticker'].unique()

# # Create plots for each ticker
# for ticker in ticker_list:
#     # Filter data for current ticker
#     ticker_data = df_price[df_price['ticker'] == ticker]
#     ticker_data.sort_values('date_time')# or 'ticker'
    
#     # Create the plot
#     plt.figure(figsize=(12, 6))
#     plt.plot(ticker_data['date_time'], ticker_data['price'], linewidth=2, marker='o', markersize=3)
    
#     # Customize the plot
#     plt.title(f'{ticker} Stock Price Over Time', fontsize=16, fontweight='bold')
#     plt.xlabel('Date', fontsize=12)
#     plt.ylabel('Price ($)', fontsize=12)
#     plt.grid(True, alpha=0.3)
#     plt.xticks(rotation=45)
    
#     # Format y-axis as currency
#     plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:.2f}'))
    
#     # Adjust layout and show
#     plt.tight_layout()
#     plt.show()

In [ ]:
SPLIT_OR_ADJ_DATES = {
    "AAPL": ["2020-08-31"],
    "AMZN": ["2022-06-06"],
    "NVDA": ["2021-07-20", "2024-06-10"],
    "WMT":  ["2024-02-26"],
    "SHW":  ["2021-04-01"],
    "HON":  ["2018-10-01", "2018-10-29"],     # spin-off related adjustments
    "MRK":  ["2021-06-03"],                    # Organon spin-off adjustment
    "MMM":  ["2024-04-01"],                    # Solventum spin-off adjustment
    "IBM":  ["2021-11-04"],                    # ex-distribution day for Kyndryl
}

rows = []
for tkr, dates in SPLIT_OR_ADJ_DATES.items():
    for d in dates:
        rows.append((tkr, d))

df_split = pd.DataFrame(rows, columns=['ticker', 'date'])
df_split['date'] = pd.to_datetime(df_split['date'])
df_split['is_split'] = 1 
df_split

In [ ]:
df_price = df_price.sort_values(['ticker', 'date'])
# close to close return on day t
df_price['return'] = df_price.groupby(['ticker'])['price'].transform(lambda x: np.log(x/x.shift(1)))
df_price = df_price.merge(df_split, on = ['ticker', 'date'], how = 'left')
df_price.loc[df_price['is_split'] == 1, 'return'] = 0 # mask out the days with stock split, change to adjusted stock split with better data 
df_price = df_price.sort_values(['ticker', 'date'])
for i in range(1, 6):
    df_price['return_lead_'+str(i)] = df_price.groupby(['ticker'])['return'].shift(-i)

Match the events with the market reaction 

In [ ]:
def _to_naive_datetime(s):
    s = pd.to_datetime(s, errors="coerce")
    if pd.api.types.is_datetime64tz_dtype(s.dtype):
        # If tz-aware, drop timezone information safely
        return s.dt.tz_convert(None)
    else:
        # If tz-naive, ensure it's explicitly naive
        return s.dt.tz_localize(None)

def next_price_after_event(df_sentiment, df_price,
                           by="ticker",
                           event_col="event_timestamp_et",
                           price_time_col="date_time"):
    # 1) Clean & normalize
    s = df_sentiment.dropna(subset=[by, event_col]).copy()
    p = df_price.dropna(subset=[by, price_time_col]).copy()

    s[by] = s[by].astype(str)
    p[by] = p[by].astype(str)

    s[event_col]     = _to_naive_datetime(s[event_col])
    p[price_time_col]= _to_naive_datetime(p[price_time_col])

    s = s.dropna(subset=[event_col])
    p = p.dropna(subset=[price_time_col])

    # 2) Global stable sort so each group's time is sorted
    s = s.sort_values([by, event_col], kind="mergesort")
    p = p.sort_values([by, price_time_col], kind="mergesort")

    # 3) Per-ticker merge_asof (no dtype headaches, no renaming)
    out = []
    price_cols = [c for c in p.columns if c != by]  # keep ticker only from the left
    for tkr, s_part in s.groupby(by, sort=False):
        p_part = p[p[by] == tkr]
        # Drop 'by' on the right to avoid duplicate ticker columns
        m = pd.merge_asof(
            s_part,
            p_part[price_cols],
            left_on=event_col,
            right_on=price_time_col,
            direction="forward",
            allow_exact_matches=False
        )
        out.append(m)

    merged = pd.concat(out, ignore_index=True)

    # 4) Report
    n_events  = len(s)
    n_matched = merged[price_time_col].notna().sum() if price_time_col in merged.columns else 0
    print(f"Successfully matched {n_matched} of {n_events} sentiment records")
    print(f"Success rate: {100.0 * n_matched / n_events:.1f}%")

    return merged

# --- Use it ---
merged_df = next_price_after_event(df_sentiment, df_price)
print("\nColumns:", merged_df.columns.tolist())
print("\nHead:\n", merged_df.head())


Look at the 1 day stock reaction (during the call) and 3 day return after the call (post earning drift), winsorize the return reaction at 1 and 99 percentile


Positive correlation between analyst sentiment and market reaction and drift

In [ ]:


# ---------------------------------------------------------
# 1) Prep: 3-day return after the event (lead1+lead2+lead3)
# ---------------------------------------------------------
merged_df = merged_df.copy()
merged_df['return_3d'] = merged_df[['return_lead_1', 'return_lead_2', 'return_lead_3']].sum(axis=1)

# ---------------------------------------------------------
# 2) Winsorize (clip) helper at [1%, 99%]
# ---------------------------------------------------------
def winsorize_df(df, cols, lower=0.01, upper=0.99):
    df = df.copy()
    for c in cols:
        q_low, q_hi = df[c].quantile([lower, upper])
        df[c] = df[c].clip(lower=q_low, upper=q_hi)
    return df

# ---------------------------------------------------------
# 3) OLS with t-stat for slope
# ---------------------------------------------------------
def ols_beta_t(x, y):
    """
    Simple OLS: y ~ a + b*x
    Returns: intercept a, slope b, t-stat of b, n
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    m = np.isfinite(x) & np.isfinite(y)
    x, y = x[m], y[m]
    n = len(x)
    if n < 3:
        return np.nan, np.nan, np.nan, n

    X = np.column_stack([np.ones(n), x])
    # OLS solution
    beta, _, _, _ = np.linalg.lstsq(X, y, rcond=None)  # [a, b]
    y_hat = X @ beta
    resid = y - y_hat
    s2 = (resid @ resid) / (n - 2)  # residual variance
    XtX_inv = np.linalg.inv(X.T @ X)
    se_b = np.sqrt(s2 * XtX_inv[1, 1])
    t_b = beta[1] / se_b if se_b > 0 else np.nan
    return beta[0], beta[1], t_b, n

# ---------------------------------------------------------
# 4) Plot helper: scatter + OLS line + beta/t annotation
# ---------------------------------------------------------
def scatter_with_ols(ax, df, x_col, y_col, title, y_label, clip=True):
    data = df[[x_col, y_col]].dropna().copy()
    if clip:
        data = winsorize_df(data, [y_col], 0.01, 0.99)

    a, b, t, n = ols_beta_t(data[x_col], data[y_col])

    ax.scatter(data[x_col], data[y_col], alpha=0.6, color = 'blue')

    if np.isfinite(b):
        x_vals = np.linspace(data[x_col].min(), data[x_col].max(), 200)
        y_vals = a + b * x_vals
        ax.plot(x_vals, y_vals, linewidth=2, label="OLS fit", color = 'orange')

    ax.set_xlabel("Sentiment Score")
    ax.set_ylabel(y_label)
    ax.set_title(title)
    ax.grid(True, alpha=0.3)
    ax.legend(loc="best")

    beta_str = f"{b:.4f}" if np.isfinite(b) else "NA"
    t_str    = f"{t:.2f}" if np.isfinite(t) else "NA"
    ax.text(0.02, 0.98, f"β = {beta_str}\nt-stat = {t_str}\nn = {n}",
            transform=ax.transAxes, va="top", ha="left",
            bbox=dict(facecolor="white", alpha=0.7, boxstyle="round"))

# ---------------------------------------------------------
# 5) Two-panel figure: 1D reaction and 3D drift
# ---------------------------------------------------------
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharex=False, sharey=False)

# Left: 1-Day Return (initial reaction)
scatter_with_ols(
    axes[0],
    merged_df,
    x_col="sentiment_score",
    y_col="return",
    title="Analyst Sentiment and Daily Return During Earnings (OLS, 1–99% Winsorized)",
    y_label="1D Return",
    clip=True
)

# Right: 3-Day Return (post-call drift)
scatter_with_ols(
    axes[1],
    merged_df,
    x_col="sentiment_score",
    y_col="return_3d",
    title="Analyst Sentiment and 3D Return after Earnings (OLS, 1–99% Winsorized)",
    y_label="3D Return",
    clip=True
)

plt.tight_layout()
plt.show()


In [ ]:
# use the change of analyst sentiment (vs last earning call)
# higher analyst sentiment (compared to last earnings call) -> higher stock return

fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharex=False, sharey=False)

# Left: 1-Day Return (initial reaction)
scatter_with_ols(
    axes[0],
    merged_df,
    x_col="sentiment_change",
    y_col="return",
    title="Sentiment(vs last earnings call) and Daily Return During Earnings (1–99% Winsorized)",
    y_label="1D Return",
    clip=True
)

# Right: 3-Day Return (post-call drift)
scatter_with_ols(
    axes[1],
    merged_df,
    x_col="sentiment_change",
    y_col="return_3d",
    title="Sentiment(vs last earnings call) and 3D Return after Earnings (1–99% Winsorized)",
    y_label="3D Return",
    clip=True
)

plt.tight_layout()
plt.show()


# ML Model for Financial Modeling

## 📌 Goal  
Test whether short-, medium-, and long-horizon return **slices** help **predict 5-day forward log returns** (with a 1-day implementation lag) using LightGBM, then deploy those forecasts in a simple **weekly long/short** strategy.

---

## 🔨 What We Did  

1. **Feature Engineering (no leakage)**  
   - Compute daily **log returns** and **clip** at the 1st/99th percentiles (applied once so X and y share identical preprocessing).  
   - Build **non-overlapping momentum slices** ending at day *t*:  
     - `r_1` (today)  
     - `r_5_1` (t-4..t-1)  
     - `r_10_5` (t-9..t-5)  
     - `r_21_10` (t-20..t-11)  
     - `r_63_21` (t-62..t-21)  

2. **Label Construction (live-consistent)**  
   - Target **y(t) = Σ logret[t+2..t+6]**.  
   - Interpretation: predict at **close t**, trade at **close t+1** (1-day lag), and realize from **t+2 through t+6**.  

3. **Walk-Forward Training (quarterly)**  
   - For each test quarter:  
     - **Train** on all quarters \< (Q-2)  
     - **Validate** on (Q-2, Q-1)  
     - **Test** on **Q**  
   - Enforce **strict cutoffs** so rows needing returns beyond the split end are dropped (**no look-ahead**).  

4. **Modeling (LightGBM)**  
   - LightGBM **regression** with early stopping.  
   - **Small grid search** each quarter over `learning_rate` and `num_leaves` for a quick bias–variance sweep.  

5. **Prediction & Portfolio (Tue → Wed)**  
   - Generate forecasts on **Tuesdays** in the test quarter.  
   - At **Wednesday close**, go **long top-5** and **short bottom-5** by predicted return (equal weight).  
   - Hold through **Thu → next Wed** (exactly the `t+2..t+6` window used for training).  
   - **Transaction cost**: 3.0 bps one-way applied via weekly turnover.  

6. **Performance Evaluation**  
   - **Forecast quality**: RMSE (and optional R²) on Tuesday predictions.  
   - **Strategy metrics**:  
     - Information Ratio (before/after costs)  
     - Turnover & cost impact (bps)  
     - Max drawdown & Calmar  
     - Equity curves (before vs after cost)  

---

## 📈 Takeaways  
- The pipeline cleanly separates **signal quality** from **execution realism** (turnover, costs, drawdowns).  
- Quarterly re-training helps LightGBM **adapt to regime shifts**.  
- Even with compact features and a tiny grid, forecasts can be useful—yet **costs matter**: at 3 bps one-way, turnover meaningfully affects realized alpha.  

---

👉 **In short:** an **end-to-end, no-leakage backtest** where LightGBM predicts near-term returns from orthogonalized momentum slices, converted into a **systematic weekly L/S portfolio** with explicit timing (Tue signal → Wed trade) and realistic cost modeling.


In [ ]:
# ============================================================
# End-to-end LightGBM demo for weekly L/S portfolio on stocks
# ------------------------------------------------------------
# 1) Build *log* returns, globally clip at 1%/99% once (X & y consistent)
# 2) Features (non-overlapping "slices" of past returns ending at t):
#       r_1, r_5_1, r_10_5, r_21_10, r_63_21
# 3) Label y(t) = sum of log returns t+2..t+6 (lag=1 day, 5-day horizon)
# 4) Train quarterly: TRAIN (<Q-2), VALID (Q-2,Q-1), TEST (Q)
#    Predict only on Tuesdays of test quarter (signals at Tue close)
# 5) Portfolio: form at Wed close (one-day lag), hold Thu→next Wed
# ============================================================

import numpy as np
import pandas as pd
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from itertools import product

# -----------------------------
# Config — horizons & portfolio
# -----------------------------
LAG_DAYS     = 2   # start at t+2 because we implement at t+1 (one-day lag)
HORIZON_DAYS = 5   # total of 5 trading days: t+2..t+6
TOP_K        = 5
BOT_K        = 5
MIN_XS       = 10  # min names on a Tuesday to form portfolio
COST_BPS     = 3.0 # one-way cost per unit turnover (bps)

# ------------------------------------
# 0) Utilities: dates & simple metrics
# ------------------------------------
def to_quarter(dates):
    return pd.PeriodIndex(pd.to_datetime(dates), freq='Q')

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def _build_bd_shift(trading_dates):
    """Business-day shifter on your union of trading dates."""
    trading_dates = pd.Index(pd.to_datetime(np.sort(pd.unique(trading_dates))))
    pos = pd.Series(np.arange(len(trading_dates)), index=trading_dates)
    def bd_shift(date, n):
        if pd.isna(date): return pd.NaT
        i = pos.get(date, None)
        if i is None: return pd.NaT
        j = i + n
        return trading_dates[j] if 0 <= j < len(trading_dates) else pd.NaT
    return bd_shift, trading_dates

# ------------------------------------------------------------
# 1) Returns prep: log returns + one-time (global) clipping
#    Why once? Keeps preprocessing identical for X and y.
# ------------------------------------------------------------
def ensure_log_returns_and_clip(df, q_low=0.01, q_high=0.99):
    """
    Input must contain: 'date','ticker', and 'close' (or 'price').
    Output adds 'return' = log(P_t/P_{t-1}) and clips it globally at
    [q_low, q_high] percentiles using the *full* sample.
    """
    need = {'date','ticker'}
    if not need.issubset(df.columns):
        raise ValueError("df must have ['date','ticker', plus 'close' or 'price'].")

    px = 'close' if 'close' in df.columns else ('price' if 'price' in df.columns else None)
    if px is None:
        raise ValueError("Provide a price column named 'close' (preferred) or 'price'.")

    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(['ticker','date'])

    # log return per ticker
    df['return'] = df.groupby('ticker', group_keys=False)[px].apply(
        lambda s: np.log(s) - np.log(s.shift(1))
    )

    # one-time global clipping (FULL sample, this is for simplicity you should do it using a rolling window in practice)
    lo, hi = df['return'].quantile([q_low, q_high])
    df['return'] = df['return'].clip(lower=lo, upper=hi)
    return df

# ----------------------------------------------------------------
# 2) Features & label
#    Features at close t use info up to & including t.
#    Label y(t) = sum of clipped log returns t+2..t+6 (default).
# ----------------------------------------------------------------
def make_fullpanel_features_labels(df,
                                   lag_days=LAG_DAYS,
                                   horizon_days=HORIZON_DAYS):
    df = ensure_log_returns_and_clip(df)
    g  = df.groupby('ticker', group_keys=False)

    # rolling sums ending at t (include t)
    r   = g['return']
    r5  = r.transform(lambda s: s.rolling(5).sum())
    r10 = r.transform(lambda s: s.rolling(10).sum())
    r21 = r.transform(lambda s: s.rolling(21).sum())
    r63 = r.transform(lambda s: s.rolling(63).sum())

    # orthogonalized slices (non-overlapping)
    df['r_1']      = r.transform(lambda s: s)          # today
    df['r_5_1']    = r5  - df['r_1']                   # days t-4..t-1
    df['r_10_5']   = r10 - r5                          # days t-9..t-5
    df['r_21_10']  = r21 - r10                         # days t-20..t-11
    df['r_63_21']  = r63 - r21                         # days t-62..t-21
    feat_cols = ['r_1','r_5_1','r_10_5','r_21_10','r_63_21']

    # label y(t): forward window starting at t+lag_days; length = horizon_days
    # We align the *end* of the window to t by shifting forward, then rolling.
    def fwd_sum_label(s):
        return s.shift(-(lag_days + horizon_days - 1)).rolling(horizon_days).sum()
    df['y'] = r.transform(fwd_sum_label)

    realized_col = f'realized_tplus{lag_days}_x{horizon_days}'
    df[realized_col] = df['y']

    # drop rows without complete features/labels
    df = df.dropna(subset=feat_cols + ['y']).reset_index(drop=True)
    
    return df, feat_cols, realized_col

# ------------------------------------------------------------
# 3) Model: tiny LightGBM grid + early stopping
# ------------------------------------------------------------
def make_param_grid(base=None):
    base = base or {}
    grid = {
        "learning_rate": [0.03, 0.05, 0.10],
        "num_leaves":    [31, 63],
    }
    fixed = {
        "objective": "regression",
        "metric": "l2",
        "min_data_in_leaf": 50,
        "feature_fraction": 0.75,
        "bagging_fraction": 0.75,
        "bagging_freq": 1,
        "lambda_l2": 1.0,
        "verbosity": -1,
        "seed": base.get("seed", 42),
    }
    out = []
    for lr in grid["learning_rate"]:
        for nl in grid["num_leaves"]:
            p = fixed.copy()
            p.update({"learning_rate": lr, "num_leaves": nl})
            out.append(p)
    return out

def train_with_hp_search(Xtr, ytr, Xva, yva,
                         base_params=None, num_boost_round=5000,
                         es_rounds=200, verbose=False):
    grid  = make_param_grid(base_params)
    best  = {"score": float("inf"), "model": None, "params": None}
    dtr   = lgb.Dataset(Xtr, label=ytr, free_raw_data=False)
    dva   = lgb.Dataset(Xva, label=yva, free_raw_data=False)

    for i, params in enumerate(grid, 1):
        model = lgb.train(
            params, dtr, num_boost_round=num_boost_round,
            valid_sets=[dva], valid_names=["valid"],
            callbacks=[early_stopping(es_rounds), log_evaluation(0 if not verbose else 50)]
        )
        score = model.best_score.get("valid", {}).get("l2", float("inf"))
        if verbose:
            print(f"[{i:>2}/{len(grid)}] valid_l2={score:.6f}  "
                  f"lr={params['learning_rate']}  leaves={params['num_leaves']}")
        if score < best["score"]:
            best.update({"score": score, "model": model, "params": params})
    return best["model"], best["params"], best["score"]

# -------------------------------------------------------------------
# 4) Quarterly loop: predict *Tuesdays* in test quarter (strict cutoff)
#     - TRAIN = all quarters before the last two (Q-2, Q-1)
#     - VALID = Q-2 and Q-1
#     - TEST  = Q
#   Strictly enforce: keep only rows with t ≤ (split_end - (lag+horizon-1)bd)
# -------------------------------------------------------------------
def quarterly_train_predict_tuesdays(df_price,
                                     lag_days=LAG_DAYS,
                                     horizon_days=HORIZON_DAYS,
                                     params=None,
                                     num_boost_round=5000,
                                     es_rounds=200,
                                     random_state=42,
                                     log_model=True):
    if params is None:
        params = dict(
            objective='regression', metric='l2',
            learning_rate=0.05, num_leaves=63,
            min_data_in_leaf=50, feature_fraction=0.9,
            bagging_fraction=0.8, bagging_freq=1,
            lambda_l2=1.0, verbosity=-1, seed=random_state
        )

    panel, feat_cols, realized_col = make_fullpanel_features_labels(
        df_price, lag_days=lag_days, horizon_days=horizon_days
    )
    panel['q'] = to_quarter(panel['date'])

    bd_shift, _ = _build_bd_shift(panel['date'])
    quarters    = sorted(panel['q'].unique())
    lookahead   = lag_days + horizon_days - 1   # = 6 with defaults

    preds_list = []

    # Need ≥ 4 train + 2 val + 1 test quarters → start at index 6
    for i in range(6, len(quarters)):
        q_test  = quarters[i]
        q_val   = quarters[i-2:i]
        q_train = quarters[:i-2]
        if len(q_train) < 4:
            continue

        train_all = panel[panel['q'].isin(q_train)].copy()
        val_all   = panel[panel['q'].isin(q_val)].copy()
        testQ     = panel[panel['q'] == q_test].copy()
        if train_all.empty or val_all.empty or testQ.empty:
            continue

        # strict cutoffs to avoid label peeking past split ends
        cutoff_train = bd_shift(train_all['date'].max(), -lookahead)
        cutoff_val   = bd_shift(val_all['date'].max(),   -lookahead)
        if pd.isna(cutoff_train) or pd.isna(cutoff_val):
            continue

        train = train_all[train_all['date'] <= cutoff_train].copy()
        valid = val_all[val_all['date']   <= cutoff_val].copy()
        if train.empty or valid.empty:
            continue

        # (No extra winsorization — we already globally clipped returns)
        Xtr, ytr = train[feat_cols], train['y']
        Xva, yva = valid[feat_cols], valid['y']

        model, best_params, best_score = train_with_hp_search(
            Xtr, ytr, Xva, yva,
            base_params=params,
            num_boost_round=num_boost_round,
            es_rounds=es_rounds,
            verbose=False
        )

        if log_model:
            # Register best model for the quarter (Dash)
            model_name = f"FIS_{q_test}"
            registry.log_model(model,
                            model_name=model_name,
                            version_name="v1",
                            sample_input_data=Xtr.head(100),
                            options={
                                "relax_version": False,
                                "target_methods": ["predict"],
                                "method_options": {
                                    "predict": {"case_sensitive": True}
                                }
                            }
                        )

        # Predict only on Tuesdays in TEST quarter
        test_tues = testQ[testQ['date'].dt.weekday == 1]  # Monday=0, Tuesday=1
        if test_tues.empty:
            continue

        X_test = test_tues[feat_cols]
        yhat   = model.predict(X_test, num_iteration=model.best_iteration)

        out = test_tues[['date','ticker','y']].copy()
        out['y_hat'] = yhat
        out.rename(columns={'date':'signal_date'}, inplace=True)

        # Attach trade date (Wednesday) and holding end (next Wednesday)
        out['trade_date'] = out['signal_date'].apply(lambda d: bd_shift(d, 1))  # Wed
        out['period_end'] = out['signal_date'].apply(lambda d: bd_shift(d, 6))  # next Wed
        out['week_key']   = out['period_end']  # key for weekly PnL aggregation
        preds_list.append(out)

    if not preds_list:
        raise ValueError("No eligible test quarters or Tuesday signals after strict cutoffs.")
    preds = pd.concat(preds_list, ignore_index=True).sort_values(['signal_date','ticker'])
    return preds, realized_col, feat_cols

# ------------------------------------------------------------------
# 5) Portfolio: Tue signal → trade Wed → realize Thu..next Wed (y)
# ------------------------------------------------------------------
def weekly_ls_portfolio_tue_wed(preds, top_k=TOP_K, bottom_k=BOT_K,
                                min_names=MIN_XS, cost_per_turn_bps=COST_BPS):
    """
    - On each Tuesday, rank by y_hat; long top_k, short bottom_k (equal-weight).
    - Form at Wednesday close; realized PnL for the "week" is y(signal_date).
    - Only rebalance on Wednesdays; turnover vs last Wednesday.
    """
    sig = preds.copy()

    picks = []
    for t_sig, g in sig.groupby('signal_date', sort=True):
        if len(g) < min_names:
            continue
        g = g.sort_values('y_hat')
        k_top = min(top_k, len(g))
        k_bot = min(bottom_k, len(g))
        longs  = g.tail(k_top).assign(side=+1.0)
        shorts = g.head(k_bot).assign(side=-1.0)
        picks.append(pd.concat([longs, shorts], axis=0))

    if not picks:
        return pd.DataFrame(), {}

    picks = pd.concat(picks, ignore_index=True)

    weekly_rows = []
    prev_w = pd.Series(dtype=float)

    for week_end, d in picks.groupby('period_end', sort=True):
        # realized next-Wed PnL for holdings selected on prior Tuesday
        r = d.set_index('ticker')['y'].astype(float).dropna()

        long_tickers  = d.loc[d['side'] > 0, 'ticker'].tolist()
        short_tickers = d.loc[d['side'] < 0, 'ticker'].tolist()
        nL, nS = len(long_tickers), len(short_tickers)
        if nL == 0 or nS == 0 or r.empty:
            continue

        # equal-weight L/S
        w = pd.Series(0.0, index=pd.Index(long_tickers + short_tickers))
        w.loc[long_tickers]  =  1.0 / nL
        w.loc[short_tickers] = -1.0 / nS

        # before-cost weekly PnL (using *log-return sums*)
        ret_before = (w.reindex(r.index).fillna(0.0) * r).sum()

        # turnover vs last Wed
        union  = w.index.union(prev_w.index)
        w_u    = w.reindex(union).fillna(0.0)
        prev_u = prev_w.reindex(union).fillna(0.0)
        turnover = 0.5 * (w_u - prev_u).abs().sum()

        cost = (cost_per_turn_bps / 1e4) * turnover
        ret_after = ret_before - cost

        weekly_rows.append({
            'week_end': week_end,
            'ret_before_cost': ret_before,
            'ret_after_cost':  ret_after,
            'turnover': turnover
        })
        prev_w = w

    weekly = pd.DataFrame(weekly_rows).sort_values('week_end').reset_index(drop=True)
    if weekly.empty:
        return weekly, {}

    # equity & drawdown
    weekly['equity_before'] = (1 + weekly['ret_before_cost']).cumprod()
    weekly['equity_after']  = (1 + weekly['ret_after_cost']).cumprod()
    peak = weekly['equity_before'].cummax()
    weekly['drawdown'] = weekly['equity_before'] / peak - 1.0

    # summary stats (annualize by 52 weeks)
    mu_b, sd_b = weekly['ret_before_cost'].mean(), weekly['ret_before_cost'].std(ddof=1)
    mu_a, sd_a = weekly['ret_after_cost'].mean(),  weekly['ret_after_cost'].std(ddof=1)
    ir_before  = (mu_b/sd_b)*np.sqrt(52) if sd_b>0 else np.nan
    ir_after   = (mu_a/sd_a)*np.sqrt(52) if sd_a>0 else np.nan
    mdd_before = weekly['drawdown'].min()
    ann_b      = mu_b * 52
    calmar_b   = (ann_b / abs(mdd_before)) if (mdd_before < 0) else np.nan

    stats = dict(
        IR_before=ir_before, IR_after=ir_after,
        max_drawdown_before=mdd_before, calmar_before=calmar_b,
        avg_turnover=weekly['turnover'].mean(),
        p90_turnover=weekly['turnover'].quantile(0.9),
        cost_bps=cost_per_turn_bps
    )
    return weekly, stats


In [ ]:
# =========================
# 6) Run the demo
# =========================
# Expect df_price with columns: ['date','ticker','close']  (or 'price')
# df_price = ... (load your panel here)

tue_preds, realized_col, feat_cols = quarterly_train_predict_tuesdays(
    df_price,
    lag_days=LAG_DAYS, horizon_days=HORIZON_DAYS
)

Interpretation: Even a simple model applied to Dow Jones component stocks can deliver a reasonable PnL, achieving an IR of around 0.5 (before cost). 


Potential improvements: \
Incorporate risk-adjusted returns for training and portfolio construction (e.g., residuals after controlling for market and/or industry exposures). \
Add more predictive signals and expand the investment universe to increase diversification and alpha potential.

In [ ]:
weekly_pnl, stats = weekly_ls_portfolio_tue_wed(
    tue_preds,
    top_k=TOP_K, bottom_k=BOT_K, min_names=MIN_XS, cost_per_turn_bps=COST_BPS
)

if weekly_pnl.empty or not stats:
    print("No weekly PnL generated. Check Tuesday coverage, MIN_XS, TOP/BOT K, and lookahead cutoffs.")
else:
    print(f"IR before cost : {stats['IR_before']:.3f}")
    print(f"IR after  cost : {stats['IR_after']:.3f}")
    print(f"Max DD (before): {stats['max_drawdown_before']:.2%}")
    print(f"Calmar (before): {stats['calmar_before']:.3f}")
    print(f"Avg turnover   : {stats['avg_turnover']:.3f}")

    # PnL curves
    plt.figure(figsize=(8,4))
    plt.plot(weekly_pnl['week_end'], weekly_pnl['equity_before'], label='Before cost')
    plt.plot(weekly_pnl['week_end'], weekly_pnl['equity_after'],  label='After cost')
    plt.xlabel('Week end (Wed)')
    plt.ylabel('Equity (compounded)')
    plt.title('Cumulative PnL: Before vs After Costs (Tue signal → Wed trade)')
    plt.legend(); plt.grid(alpha=0.3); plt.show()

    # Drawdown (before cost only)
    plt.figure(figsize=(8,4))
    plt.plot(weekly_pnl['week_end'], weekly_pnl['drawdown'])
    plt.xlabel('Week end (Wed)')
    plt.ylabel('Drawdown')
    plt.title('Drawdown Curve (Before Cost)')
    plt.grid(alpha=0.3); plt.show()